<a href="https://colab.research.google.com/github/hossainsiam133/AgriGrow/blob/main/Code/AgriGrow_Talha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello")

Hello
